In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sat Nov 13 14:19:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
%%bash
pip install ctc-segmentation
git clone --recursive https://github.com/parlance/ctcdecode.git
git clone https://github.com/farisalasmary/wav2vec2-kenlm.git

Submodule path 'third_party/ThreadPool': checked out '9a42ec1329f259a5f4881a291db1dcb8f2ad9040'
Submodule path 'third_party/kenlm': checked out '35835f1ac4884126458ac89f9bf6dd9ccad561e0'


Cloning into 'ctcdecode'...
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/wav2vec2-kenlm/ctcdecode/third_party/ThreadPool'...
Cloning into '/content/wav2vec2-kenlm/ctcdecode/third_party/kenlm'...
Cloning into 'wav2vec2-kenlm'...


In [ ]:
%cd ctcdecode/

/content/wav2vec2-kenlm/ctcdecode


In [ ]:
!pip install .

Processing /content/wav2vec2-kenlm/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp37-cp37m-linux_x86_64.whl size=13267271 sha256=b04580551205551035abad185f12cdef7dead5d3bef4414857296a8984d74887
  Stored in directory: /tmp/pip-ephem-wheel-cache-ze40o7oc/wheels/dd/2d/d8/9b4a7a57c2ad0d1a80ed203eddfad2bd7dc58b7eef3d413ae1
Successfully built ctcdecode
  Attempting uninstall: ctcdecode
    Found existing installation: ctcdecode 1.0.3
    Uninstalling ctcdecode-1.0.3:
      Successfully uninstalled ctcdecode-1.0.3


In [ ]:
!pip3 install transformers datasets torchaudio

     |████████████████████████████████| 290 kB 4.0 MB/s 
     |████████████████████████████████| 2.9 MB 86.7 MB/s 
     |████████████████████████████████| 243 kB 87.5 MB/s 
     |████████████████████████████████| 1.1 MB 54.3 MB/s 
     |████████████████████████████████| 132 kB 94.7 MB/s 
     |████████████████████████████████| 271 kB 85.7 MB/s 
     |████████████████████████████████| 192 kB 71.7 MB/s 
     |████████████████████████████████| 160 kB 89.1 MB/s 


In [ ]:
%cd ..

/content/wav2vec2-kenlm


In [ ]:
!wget https://kaldi-asr.org/models/5/4gram_big.arpa.gz
!gzip -d 4gram_big.arpa.gz

--2021-11-13 14:02:51--  https://kaldi-asr.org/models/5/4gram_big.arpa.gz
Resolving kaldi-asr.org (kaldi-asr.org)... 46.101.158.64
Connecting to kaldi-asr.org (kaldi-asr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99823907 (95M) [application/x-gzip]
Saving to: ‘4gram_big.arpa.gz’

4gram_big.arpa.gz   100%[===================>]  95.20M  9.66MB/s    in 12s     

2021-11-13 14:03:04 (8.04 MB/s) - ‘4gram_big.arpa.gz’ saved [99823907/99823907]



In [ ]:
import torch
from datasets import load_dataset
from decoder import *
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Wav2Vec2CTCTokenizer
import utils

In [ ]:
MODEL_ID = "jonatasgrosman/wav2vec2-large-xlsr-53-english"

print(f'Loading Wav2Vec2CTC Model: "{MODEL_ID}"')
processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Loading Wav2Vec2CTC Model: "jonatasgrosman/wav2vec2-large-xlsr-53-english"


/usr/local/lib/python3.7/dist-packages/transformers/configuration_utils.py:341: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "


In [ ]:
vocab_dict = processor.tokenizer.get_vocab()
sort_vocab = sorted((value, key) for (key,value) in vocab_dict.items())

# Lower case ALL letters
vocab = []
for _, token in sort_vocab:
    vocab.append(token.lower())

vocab[vocab.index(processor.tokenizer.word_delimiter_token)] = ' '

In [ ]:
lm_path = '4gram_big.arpa'

In [ ]:
alpha=0
beta=0
beam_width = 1024

beam_decoder = BeamCTCDecoder(vocab, lm_path=lm_path,
                                 alpha=alpha, beta=beta,
                                 cutoff_top_n=40, cutoff_prob=1.0,
                                 beam_width=beam_width, num_processes=16,
                                 blank_index=vocab.index(processor.tokenizer.pad_token))


In [ ]:
greedy_decoder = GreedyDecoder(vocab, blank_index=vocab.index(processor.tokenizer.pad_token))

In [ ]:
audio_files_paths = ['english_sample.wav'] * 3

In [ ]:
print(f'Load audio files: "{audio_files_paths}"')
batch_audio_files, sampling_rate = utils.load_audio_files(audio_files_paths)

Load audio files: "['english_sample.wav', 'english_sample.wav', 'english_sample.wav']"


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print('Get logits from the Wav2Vec2ForCTC model....')
logits, max_signal_length = utils.get_logits(batch_audio_files, model, processor, device)

Get logits from the Wav2Vec2ForCTC model....


/usr/local/lib/python3.7/dist-packages/transformers/feature_extraction_utils.py:158: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:201.)
  tensor = as_tensor(value)


RuntimeError: ignored